## Lecture 10

## Python Parallel Computing - Part 01

### Apr 5, 2023


Part of this lecture is based on this material from previous years: https://nyu-cds.github.io/python-mpi/




You will need **mpi4py** 

To install: [https://mpi4py.readthedocs.io/en/stable/install.html](https://mpi4py.readthedocs.io/en/stable/install.html)


Run the following in terminal: 

1. sudo apt install libopenmpi-dev

2. pip install mpi4py


or 

1. brew install open-mpi

2. pip install mpi4py




### Two basic approaches

<img src="./figs/shared_memory.png" alt="shared_memory" style="width: 450px;"/>


- this is on the high level how most single computers are organized. 

<img src="./figs/distributed_memory.png" alt="distributed_memory" style="width: 500px;"/>

- on the cluster 
- just multiple indepednt computer
- than those comuters have some way to comunicate efficently 
- many parameters but within each parameter small operations  
- nyu has some gpu clusters.

## Parallelization and Amdahl's law

* Want to leverage parallelization as much as possible
* Often we cannot obtain perfect (linear) speedups, e.g., communication or global logic
* Amdahl's law is a simple law to get an idea of the speedup:
    - $N$: number of processors
    - $P$: fraction of program that can be parallelized

$$
speedup = \frac{1}{(1 - P) + \frac{P}{N}}
$$

<img src="https://nyu-cds.github.io/python-mpi/fig/01-amdahls-law.png" alt="amdahls" style="width: 300px;"/>

- this is the spped up we will see from parallelsm. 
- p is the fraction of the program that can be parallized. 
- this is the proprtion that can not be parallelized. 


---

### __MPI__ (Message Passing Interface) 

- Widely used standard


- For programming **distributed-memory**, **multiple instruction**--**multiple data** (MIMD) systems


#### __Point to point Communication__

Processes should coordinate their activities by explicitly sending and receiving messages

MPI operates as follows:
- Process A decides a message needs to be sent to process B.
- Process A packs up all of its necessary data into a buffer for process B.
- Process A indicates that the data should be sent to process B by calling the _Send_ function.
- Process B needs to acknowledge it wants to receive the message by calling the _Recv_ function.

Every time a process sends a message, there must be a process that also indicates it wants to receive the message, therefore, calls to _Send_ and _Recv_ are always paired.



<img src="./figs/send_receive.png" alt="distributed_memory" style="width: 400px;"/>

- need a way to comunicate between processes
- exhange messages
- distributed memory cpu share same peac oof memory 
- muliple insturctions instructions acros smultiple jobs
- multiple data 
point to point process a and b exhanfe memory 
- as oposed to other methods, want to specify a reciver 
- this reports a reciver that send mesasges to some other reciver 
- point to point 
- porcess a says they are going to send  amessage process b says they except to get a message from process b
- packs up all the data put it in a buffer, which is a piece of memory 
- want to send info to b 
- b syas ok i expcet a message form a specfically
- a could say potetinally sending mesasge from anyone 
-b says expecting mesage from anyone 
- the send and recive are always paire dtoghter. 
-  send an recive shoudl awlways be paried up 


### The number of processes 

- Is **fixed** when an MPI program is started 

- Each of the processes is assigned a unique integer starting from 0. 

- This integer is know as the **rank** of the process and is how each process is identified when sending and receiving messages (we will refer to rank K process as "process K").

- **MPI processes** are arranged in logical collections known as **communicators**. 

- There is one special communicator (**MPI.COMM_WORLD**) that exists when an MPI program starts, which contains all the processes in the MPI program. 


- MPI provides a few **methods** on a communicator:


> Get_size() - returns the total number of processes contained in the communicator (the size of the communicator).

> Get_rank() - returns the rank of the calling process within the communicator, between 0 and (size-1)

> Send() - sends content to a process

> Recv() - receives content from a process



- always ahve to secpficy number of core or processors. 
- number of cores are always fixed, eahc of the process gets a unique integer called a rank. n=4 means we have 4 processes. 
- rank =0,1,2,3 if n=4
- comunicator is a process 
- some comuncator needs to be in carhge of the whole comuncation, it needs to be the cordinator or controll that send and recives messages to make srue the worker nodes are doing there jobs. 
- get size, get rank, are just get attrbute functions 
- the most important thing is need to specficy number of cores or prcessors to use 
- each process has a unique rank. 

In [1]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank() 
print('hello world: size = %d, rank = %d' % (size, rank))

ImportError: libmpi.so.12: cannot open shared object file: No such file or directory

In [2]:
%%writefile mpi1.py
#####
# writing the code in the mpi1.py file
#####

from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank() ## writing this to a file
print('hello world: size = {}, rank = {}'.format(size, rank))

Writing mpi1.py


In [3]:
#####
# running MPI from the terminal with n=4 processes
# does not work in notebook for me for some reason, if so you can run in the terminal
#####

!mpiexec -n 4 python mpi1.py ## number of corres we use is 4. 12 cores -> 4 cores about good. number should be smaller than the number of cores. 

---
### One MPI program, multiple MPI processes
Making each process to perform a different computation 

In [4]:
%%writefile mpi2.py

from mpi4py import MPI
rank = MPI.COMM_WORLD.Get_rank()

a = 8.0
b = 4.0

print('Process rank',rank)

if rank == 0: ## difrent rnaks have difrent jobs. 
        print("addition:", a + b)

if rank == 1:
        print("multiplication:", a * b)

if rank == 2:
        print("maximum:", max(a,b))
        
if rank == 3:
        print("doing nothing:")
## makes sense. 

## doing things indepedntly but there is no comuncation. 

Writing mpi2.py


In [ ]:
!mpiexec -n 4 python mpi2.py

---
### Point-to-point communication
Message passing involves two processes: a **sender** and a **receiver** (commands _Send_ and _Recv_).

In [5]:
%%writefile mpi3.py
#####
# Sending a message from one process to another
#####
import numpy

from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank() 

randNum = numpy.zeros(1) 

if rank == 1:
        print("part of Process", rank, "- before receiving has the number", randNum[0])
        # generates a numpy array with one element unif. distr. from [0,1)
        randNum = numpy.random.rand(1) ## gets random number 
        print("part of Process", rank, "- drew the number", randNum[0])
        comm.Send(randNum, dest=0) ## send this to rank 0
        
if rank == 0:
        print("part of Process", rank, "- before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1) ## recive the ranom number from 1 
        print("part of Process", rank, "- received the number", randNum[0]) ## prints that rnadom number 

Writing mpi3.py


In [ ]:
!mpiexec -n 2 python mpi3.py

In [6]:
%%writefile mpi4.py
#####
# Sending a message to a process and receiving a message back
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.rand(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=0)
        print("Process", rank, "received the number", randNum[0], "from process 0")
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1)
        print("Process", rank, "received the number", randNum[0], "from process 1")
        randNum *= 20
        comm.Send(randNum, dest=1) 

Writing mpi4.py


<img src="./figs/send_receive_mul2.png" style="width: 400px;"/>

- plots that we just did preetty much. 

In [ ]:
!mpiexec -n 2 python mpi4.py


The receiving process does not always need to specify the source when issuing a Recv.

Instead, the process can accept **any message** that is being sent by another process. This is done by setting the source to **MPI.ANY_SOURCE**.

In [8]:
%%writefile mpi5.py
#####
# Sending a message to a process and receiving a message back from MPI.ANY_SOURCE
#####

import numpy
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        comm.Send(randNum, dest=0)
        comm.Recv(randNum, source=MPI.ANY_SOURCE)
        print("Process", rank, "received the number", randNum[0])
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=MPI.ANY_SOURCE) ## recivees from any source, in this case it does not matter but if had more cores. it may have difrent resuls
        print("Process", rank, "received the number", randNum[0])    
        randNum *= 2
        comm.Send(randNum, dest=1)
## this is 

Overwriting mpi5.py


- this is a nice becuase it does not nesscarilyl assume any core. 
-  any soruc emena sodnt have to specficy a specfic destiation can make sender adn reciver nodes more genrally 

In [ ]:
!mpiexec -n 2 python mpi5.py

---

Sometimes there are cases when a process might have to **send many different types of messages to another process**. Instead of having to go through extra measures to differentiate all these messages, MPI allows senders and receivers to also **specify message IDs (known as tags)** with the message. The receiving process can then request a message with a certain tag number and messages with different tags will be buffered until the process requests them.

```python
Comm.Send(buf, dest=0, tag=0)
Comm.Recv(buf, source=0, tag=0, status=None)
```

The _status_ can provide useful information
```python
info = MPI.Status()
source = info.Get_source()
tag = info.Get_tag()
count = info.Get_elements()
size = info.Get_count()
```

- tag is like a message identfier, want to make sure data 1 recives data 1 and data 2 recives data 2. 
- makes sure messages are mapped to write worker node. 
- tags shuld be numbers but dont nessacarilyl haveo to b consecutive. 

In [9]:
%%writefile mpi_tag.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank() 

data1 = None
data2 = None

if rank == 0:
    data1 = ('a','b', 'c', 'd')
    data2 = (1, 2, 3, 4)
    
    comm.send(data1, dest=1, tag=0)    
    comm.send(data2, dest=1, tag=1)
    
    

elif rank == 1:
    print('On Process',rank,'before recv: data1 = ', data1)
    print('On Process',rank,'before recv: data2 = ', data2)
    
    data1 = comm.recv(source=0, tag=0)  
    data2 = comm.recv(source=0, tag=1)
    
    print('On Process',rank,'after  recv: data1 = ', data1)
    print('On Process',rank,'after  recv: data2 = ', data2)
    

Writing mpi_tag.py


In [ ]:
!mpiexec -n 2 python mpi_tag.py

In [12]:
%%writefile mpi_status.py
#####
# Sending a message from one process to another
#####


import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

info = MPI.Status() ## ubtukuze stats function prints info 
print("info: ", info)

randNum = numpy.zeros(1) ## stuf fup gere sis run ever time. 

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0]) ## just drwas and send a random number,
        comm.Send(randNum, dest=0) 

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        comm.Recv(randNum, source=1, status=info)
        print("Process", rank, "received the number", randNum[0], "from Process", info.Get_source()) ## recives the info then, also writes stats into inof and prints out the source. 
        ## if you have more core can tell where something came from if something crashed. 

Overwriting mpi_status.py


In [11]:
!mpiexec -n 2 python mpi_status.py


### Non-blocking Communication

In the previous examples, the sender and receiver are not able to perform any action when sending or receiving a message. This can waste computation time while waiting for the call to complete. 

__Non-blocking communcation__ avoids this issue by using the _Isend_ and _Irecv_ methods, which start to send and receive operations and _then return immediately to continue computation_.

The completion of a send or receive operation can be managed using the _Test_, _Wait_, and _Cancel_ methods.


- in previous stuff if calll send but not recive we get stuck that is called blocking comuncation 
- then we can have non-blocking comuncation, which send even if soemthing is not recives.  have isend and irecive functions. 
- wait just makees the process wait for a second kinda like a blocking thing. 
- test keeps trying until it is sucessfull 
- 

In [25]:
%%writefile mpi6.py

# this code is similar to mpi3.py, 
# but it uses Wait to block the processes
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1)

if rank == 1:
        randNum = numpy.random.random_sample(1)
        print("Process", rank, "drew the number", randNum[0])
        
        req = comm.Isend(randNum, dest=0)
       # req.Wait() ## just having this one will make it 100% wrong. 
        ## wait only really makes sense when we are reciving something. when ever things are done asynchonalsly want to make sure you are not modifying hte same piece of data at teh same time.
        print('something here')
        
if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        
        req = comm.Irecv(randNum, source=1) ## might not be recived it will just move on. this gives you flexability but want to be carefull. 
        #req.Wait() ## if wait here that will not move on with out it. 
        while not req.Test():
                print("waiting")
        print("Process", rank, "received the number", randNum[0])

Overwriting mpi6.py


In [19]:
!mpiexec -n 2 python mpi6.py

### Overlap communication


**Example:** Process 1 overlaps a computation with sending the message and receiving the reply. The computation divides randNum by 10 and prints the result.

In [27]:
%%writefile mpi7.py
#####
# overlap communication
#####

import numpy
from mpi4py import MPI
comm = MPI.COMM_WORLD
rank = comm.Get_rank()

randNum = numpy.zeros(1) 

if rank == 1:
#         randNum = numpy.random.random_sample(1)
        randNum = numpy.array([50], dtype=numpy.float64) ## makes the number specfic instead of random
        print("Process", rank, "drew the number", randNum[0]) ## drew the number which will always b 50
        
        comm.Isend(randNum, dest=0) ## i send this to zero 
         
        randNum[0] /= 10 # overlap communication ## do some data modivaction not caring about what rank data has recived
        print("Process", rank, "number in overlap communication =", randNum[0]) ## 
         
        req = comm.Irecv(randNum, source=0)
        req.Wait()
        print("Process", rank, "received the number", randNum[0]) ## should out put the input devided by 5

if rank == 0:
        print("Process", rank, "before receiving has the number", randNum[0])
        req = comm.Irecv(randNum, source=1) ## i recive from the other thing 
        req.Wait()
        print("Process", rank, "received the number", randNum[0]) 
        randNum *= 2 ## then we multiply by 2 qne w3ne it back
        comm.Isend(randNum, dest=1)

Overwriting mpi7.py


In [ ]:
!mpiexec -n 2 python mpi7.py